In [4]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import time

import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

from gensim.summarization.summarizer import summarize

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
#Блок предобработки текста

stop_words = stopwords.words('english')
stop_words = stop_words + ['china', 'chinese','china\'s','china’s'] #в стоп список включаем слова, которые являются перекрестными в разных темах
lemmatizator = WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in words if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return ' '.join(words)#words  #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста. 

In [14]:

def china_summary_reuters(depth):
    """
    Подготовка резюме по основным новостям по Китаю
    depth - количество заголовков для просмотра на странице с последними новостями по Китаю
    """
      
    #Выгружаем все свежие заголовки про Китай
    r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
    #r1 = requests.get('https://www.reuters.com/search/news?blob=metals')
    coverpage =  r1.content
    soup = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news_r = soup.find_all('h2', class_='FeedItemHeadline_headline')

    #Опционально можно вывести отдельно все заголовки:
#     for i in range(0, len(coverpage_news_r)):
#         print(coverpage_news_r[i].get_text())
#         print('///')

    # Опционально можно парсить все статьи по существующим заголовкам без ограничения глубины
    #number_of_articles = len(coverpage_news_r)
    
    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []

    #замеряем время
    start_time = time.time()
    print('---Формируется подборка статей---')

    #формируем массив статей
    for n in range(0, depth):


        # Getting the link of the article
        link = coverpage_news_r[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news_r[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('div', class_='StandardArticleBody_body')
        try:
            x = body[0].find_all('p')
        except:
            pass
        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)
    print("--- Подборка сформирована. Затрачено %s секунд ---" % (time.time() - start_time))
    
    #Проходим по массиву выгруженных новостей и выполняем предобработку
    print('---Предобработка---')
    prepared_texts = []
    for item in news_contents:
        prepared_texts.append(prepare_text(item))
    print('---Предобработка выполнена---')
    print('///')
    #задаем ключевые слова по темам
    topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR','trade']

    # отбор новостей по ключевым словам и суммаризация
    i = 0
    for item in prepared_texts:
        counter = 0
        for word in item.split(' '):
            if word in topics_econ:
                counter += 1
        if counter > 2:
            print('Economic story %s' % i)
            print(summarize(item, word_count=60))
            print('Ссылка на статью:', list_links[i])
            print('///')
            i += 1


In [15]:
china_summary_reuters(5)

---Формируется подборка статей---
--- Подборка сформирована. Затрачено 2.1223766803741455 секунд ---
---Предобработка---
---Предобработка выполнена---
///
Economic story 0
bengaluru (reuters) yuan slip year-end deeper last u.s. dollar 2008 global financial crisis authority nudge partly managed currency u.s.-china trade rumble reuters poll showed.
news emerging market currencies.” analyst divided expected move u.s.-china trade relations, anticipating deterioration continuation status rest forecast improvement.
“unless interim trade deal materializes, think economic monetary easing pressure weaken, even though likely paced pboc,” said khoon goh, head asia research singapore, referring people’s bank china.
Ссылка на статью: https://www.reuters.com/article/us-usa-trump-whistleblower/trump-publicly-asks-china-to-probe-biden-even-amid-impeachment-inquiry-idUSKBN1WI0BC
///
Economic story 1
washington (reuters) u.s. president donald trump’s surprising request thursday start investigation democ